In [1]:
import pandas as pd

from collections import Counter

import re

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

from nrclex import NRCLex

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.stem import PorterStemmer

import spacy

from langdetect import detect

#import nlp

import langid

import collections

from langdetect.lang_detect_exception import LangDetectException

import locationtagger

from wordcloud import WordCloud

import gensim

from gensim.utils import simple_preprocess

import nltk

from nltk.corpus import stopwords

from gensim.models.ldamulticore import LdaMulticore

import gensim.corpora as corpora

import gensim.corpora as MmCorpus

from gensim import models

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

from googletrans import Translator

In [2]:
df = pd.read_csv('/Users/beltran/Desktop/influence_marketing/influence_marketing_reco/datasets/mergeado_df.csv')

In [3]:
df

,ID,followers,user_Description,likes,post_Description,location,locations2,hashtags,post_Url,engagement_rate
0,1,824122,⭕️nlyFitGirls\nBlogger\n12 years sharing the w...,26022.720,"@anabra7\n#OnlyFitGirls,@daniellebrandon7\n#On...",[],[],"['onlyfitgirls', 'onlyfitgirls', 'onlyfitgirls...","https://www.instagram.com/p/CqbVLmrjEKV/,https...",0.031576
1,2,898694,Öznur Uslu\n🇹🇷Türkiye-🇫🇷 Fransa\nYeni ve deta...,6380.100,Video’mu görenlerden ricam yoruma bir ❤️ koyab...,[],[],"['kiş', 'quiche']","https://www.instagram.com/p/ClWkdFID3TI/,https...",0.007099
2,3,214440,𝟭𝟬𝟬 𝗠𝗢𝗡𝗧𝗔𝗗𝗜𝗧𝗢𝗦\nDepende de ti cómo te lo monte...,319.900,"¿Cuál es el tuyo?\n*Todos llevan kiko crushed,...",[],[],[],"https://www.instagram.com/p/Cqi3186NCvJ/,https...",0.001492
3,4,893972,12 STOREEZ\nTienda de ropa\nКоллекция для мужч...,7082.600,"Мы уверены: гардероб — это больше, чем просто ...","['Россия', 'Россия', 'Italia', 'France']",[],[],"https://www.instagram.com/p/CqxI42KMV1Y/,https...",0.007923
4,5,663592,GPMuthu 24 🔘\nArtist\nBiggboss S6\nCookuwithco...,127999.976,"@kuraishi_the_entertainer,சமயபுரம் மாரியம்மன் ...",['India'],[],[],"https://www.instagram.com/p/CqVM8-APGCs/,https...",0.192890
...,...,...,...,...,...,...,...,...,...,...
2221,2222,51954,Zubaira Tukhugov\nUFC Fighter 19-4 Record\n•\n...,35503.232,Нереальный вкус @gorillaenergy @gorillafightin...,[],[],"['дубай', 'gorillaenergy', 'gorillafighting']","https://www.instagram.com/p/CqazvpjMM5l/,https...",0.683359
2222,2223,5749589,zubby michael\nActor\nhttps://youtu.be/drTnOhf...,251222.500,Midnight cruise 🔥GOD is real #ZM #A1 #doings #...,[],[],"['zm', 'a1', 'doings', 'nawedeyhere', 'blessup...","https://www.instagram.com/p/CqMJCMXjlCF/,https...",0.043694
2223,2224,408734,Zvezde Granda\nTV Programme\ngrand.nova.rs,28208.856,Sa kim to @voja.nedeljkovic obara ruke u Zvezd...,[],[],"['zvezdegranda', 'zvezdegranda']","https://www.instagram.com/p/CqV3wt4KAq-/,https...",0.069015
2224,2225,894950,Aziz Zyzz shavershian\nComunidad\n| motivation...,16446.000,"nan,Zyzz last vlog\n.\n.\n👉 @zyzzmad Follow : ...",[],[],"['zyzzpose', 'zyzz', 'zyzzmotivation', 'zyzzle...","https://www.instagram.com/p/CoppL0cIqJm/,https...",0.018376


In [4]:
def preprocess_text(text):

    stop_words = ['english', 'spanish', 'portuguese', 'french', 'arabic', 'indonesian', 'russian', 'italian']
    
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words(stop_words)]

    stemmer=PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [5]:
df['user_Description'] = df['user_Description'].fillna('')
df['post_Description'] = df['post_Description'].fillna('')

In [6]:
df['user_Description'] = df['user_Description'].apply(preprocess_text)
df['post_Description'] = df['post_Description'].apply(preprocess_text)
df

,ID,followers,user_Description,likes,post_Description,location,locations2,hashtags,post_Url,engagement_rate
0,1,824122,⭕️nlyfitgirl blogger 12 year share work athlet...,26022.720,"@ anabra7 # onlyfitgirl , @ daniellebrandon7 #...",[],[],"['onlyfitgirls', 'onlyfitgirls', 'onlyfitgirls...","https://www.instagram.com/p/CqbVLmrjEKV/,https...",0.031576
1,2,898694,öznur uslu 🇹🇷türkiye-🇫🇷 fransa yeni detaylı v...,6380.100,video ’ mu görenlerden ricam yoruma bir ❤️ koy...,[],[],"['kiş', 'quiche']","https://www.instagram.com/p/ClWkdFID3TI/,https...",0.007099
2,3,214440,𝟭𝟬𝟬 𝗠𝗢𝗡𝗧𝗔𝗗𝗜𝗧𝗢𝗦 depend cómo mont 🔥 tiktok : 100...,319.900,"¿cuál ? * llevan kiko crush , sitio confianza ...",[],[],[],"https://www.instagram.com/p/Cqi3186NCvJ/,https...",0.001492
3,4,893972,12 storeez tienda ropa коллекция мужчин : @ 12...,7082.600,"уверены : гардероб — это , просто набор вещей ...","['Россия', 'Россия', 'Italia', 'France']",[],[],"https://www.instagram.com/p/CqxI42KMV1Y/,https...",0.007923
4,5,663592,gpmuthu 24 🔘 artist biggboss s6 cookuwithcomal...,127999.976,"@ kuraishi_the_entertain , சமயபுரம் மாரியம்மன்...",['India'],[],[],"https://www.instagram.com/p/CqVM8-APGCs/,https...",0.192890
...,...,...,...,...,...,...,...,...,...,...
2221,2222,51954,zubaira tukhugov ufc fighter 19-4 record • воп...,35503.232,нереальный вкус @ gorillaenergi @ gorillafight...,[],[],"['дубай', 'gorillaenergy', 'gorillafighting']","https://www.instagram.com/p/CqazvpjMM5l/,https...",0.683359
2222,2223,5749589,zubbi michael actor http : //youtu.be/drtnohfzafa,251222.500,midnight cruis 🔥god real # zm # a1 # do # nawe...,[],[],"['zm', 'a1', 'doings', 'nawedeyhere', 'blessup...","https://www.instagram.com/p/CqMJCMXjlCF/,https...",0.043694
2223,2224,408734,zvezd granda tv programm grand.nova.r,28208.856,kim @ voja.nedeljkov obara ruke u zvezdama gra...,[],[],"['zvezdegranda', 'zvezdegranda']","https://www.instagram.com/p/CqV3wt4KAq-/,https...",0.069015
2224,2225,894950,aziz zyzz shavershian comunidad | motiv post e...,16446.000,"nan , zyzz last vlog . . 👉 @ zyzzmad follow : ...",[],[],"['zyzzpose', 'zyzz', 'zyzzmotivation', 'zyzzle...","https://www.instagram.com/p/CoppL0cIqJm/,https...",0.018376


In [7]:
df['all_descriptions'] = df['user_Description'].str.cat(df['post_Description'], sep=' ')

In [8]:
translator = Translator()
descripciones = []

for index, row in df.iterrows():
    text = row['all_descriptions']
    if text:
        try:
            language = detect(text)
            if language != 'en':
                translated = translator.translate(text, src=language, dest="en")
                descripcion = str(translated.text)
                descripciones.append(descripcion)
            else:
                descripciones.append(text)
        except:
            # if there is an error in translation, we can leave the original text
            descripciones.append(text)
    else:
        descripciones.append(text)

df['traducido'] = descripciones

In [9]:
def count_words(text):
    words = re.findall('\w+', text.lower())
    return Counter(words)

In [10]:
all_text = ' '.join(df['all_descriptions'])

In [11]:
word_count = count_words(all_text)

In [12]:
sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
len(sorted_words)

122789

In [13]:
mywords = []
for word, count in sorted_words[:35]:
    print(word)
    mywords.append(word)

love
com
1
link
2
bio
new
u
day
one
s
make
thank
get
time
3
like
yang
year
happi
follow
look
video
beauti
life
2023
go
nan
4
see
5
www
use
work
first


In [14]:
def delete_words(text):
    for word in mywords:
        if word in text:
            text = text.replace(word, '')
    return text

# Apply the function to the column
df['all_descriptions'] = df['all_descriptions'].apply(lambda x: delete_words(x))

In [15]:
import os

folder_path = '/Users/beltran/Desktop/influence_marketing/influence_marketing_reco/datasets/categories'

files = [os.path.splitext(f)[0] for f in os.listdir(folder_path)]

print(files)

['fitness_sport', 'music', 'gaming', 'finance_investing', 'belleza', 'luxury_lifestyle', 'cars_motorcycles', 'science', 'realEstate', 'travel', 'humor_memes', 'art_design', 'food_cooking', 'fashion', 'education', 'politics', 'social', 'culture', 'technology', 'parenting_family', 'business', 'pets', 'nature']


In [16]:
import os

folder_path = '/Users/beltran/Desktop/influence_marketing/influence_marketing_reco/datasets/categories'
lists_dict = {}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        list_name = os.path.splitext(filename)[0]
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line.strip() for line in f.readlines()]
            lists_dict[list_name] = lines

print(lists_dict)

for key in lists_dict.keys():
    print(key)

{'fitness_sport': ['health', 'suit', 'accommodate', 'fulfill', 'go', 'workout', 'agree', 'fitness', 'lifestyle', 'set', 'suited', 'meet', 'strength', 'be', 'fulfil', 'gym', 'suitable', 'motorize', 'check', 'satisfy', 'appropriate', 'running', 'physique', 'match', 'exercise', 'sports', 'conform to', 'tantrum', 'able', 'dovetail', 'nutrition', 'ready', 'healthy', 'shape', 'trainer', 'diet', 'parallel', 'adapt', 'fun', 'muscle', 'weight', 'athlete', 'wellness', 'correspond', 'body', 'lifting', 'burst', 'acceptable', 'activity', 'look', 'fill', 'conditioning', 'cardio', 'bodybuilder', 'weightlifting', 'harmonise', 'athletics', 'calisthenics', 'aesthetic', 'nutritional', 'consort', 'concord', 'dance', 'adjust', 'provide', 'yoga', 'sport', 'harmonize', 'gyms', 'supply', 'living', 'training', 'refit', 'food', 'answer', 'form', 'smartwatch', 'conditioned', 'endurance', 'athletic', 'books', 'able-bodied', 'weightloss', 'performance', 'games', 'boxing', 'size', 'attractiveness', 'urban', 'in con

In [17]:
max_categories = []

# loop through each row of the dataframe and calculate the category with the highest count
for index, row in df.iterrows():
    # create a dictionary to store the counts for each category
    counts = {key: 0 for key in lists_dict.keys()}
    # loop through each word in the text and increment the counts if the word matches a category
    for word in row['traducido'].split():
        for category, words in lists_dict.items():
            if word.lower() in words:
                counts[category] += 1
    # find the category with the highest count
    max_count = max(counts.values())
    max_category = [category for category, count in counts.items() if count == max_count][0]
    # append the max category to the list
    max_categories.append(max_category)

# add the max category column to the dataframe
df['max_category'] = max_categories

In [18]:
df

,ID,followers,user_Description,likes,post_Description,location,locations2,hashtags,post_Url,engagement_rate,all_descriptions,traducido,max_category
0,1,824122,⭕️nlyfitgirl blogger 12 year share work athlet...,26022.720,"@ anabra7 # onlyfitgirl , @ daniellebrandon7 #...",[],[],"['onlyfitgirls', 'onlyfitgirls', 'onlyfitgirls...","https://www.instagram.com/p/CqbVLmrjEKV/,https...",0.031576,⭕️nlyfitgirl blogger hare athlet photograph...,⭕️nlyfitgirl blogger 12 year share work athlet...,finance_investing
1,2,898694,öznur uslu 🇹🇷türkiye-🇫🇷 fransa yeni detaylı v...,6380.100,video ’ mu görenlerden ricam yoruma bir ❤️ koy...,[],[],"['kiş', 'quiche']","https://www.instagram.com/p/ClWkdFID3TI/,https...",0.007099,öznr l 🇹🇷türkiye-🇫🇷 frana yeni detaylı l tari...,öznur uslu 🇹🇷türkiye-🇫🇷 fransa yeni detaylı v...,fitness_sport
2,3,214440,𝟭𝟬𝟬 𝗠𝗢𝗡𝗧𝗔𝗗𝗜𝗧𝗢𝗦 depend cómo mont 🔥 tiktok : 100...,319.900,"¿cuál ? * llevan kiko crush , sitio confianza ...",[],[],[],"https://www.instagram.com/p/Cqi3186NCvJ/,https...",0.001492,𝟭𝟬𝟬 𝗠𝗢𝗡𝗧𝗔𝗗𝗜𝗧𝗢𝗦 depend cómo mont 🔥 tiktok : 00m...,𝟭𝟬𝟬 𝗠𝗢𝗡𝗧𝗔𝗗𝗜𝗧𝗢𝗦 Depend how Mont 🔥 Tiktok: 100mo...,social
3,4,893972,12 storeez tienda ropa коллекция мужчин : @ 12...,7082.600,"уверены : гардероб — это , просто набор вещей ...","['Россия', 'Россия', 'Italia', 'France']",[],[],"https://www.instagram.com/p/CqxI42KMV1Y/,https...",0.007923,toreez tienda ropa коллекция мужчин : @ toree...,12 Storeez Tienda Ropa Collection of men: @ 12...,parenting_family
4,5,663592,gpmuthu 24 🔘 artist biggboss s6 cookuwithcomal...,127999.976,"@ kuraishi_the_entertain , சமயபுரம் மாரியம்மன்...",['India'],[],[],"https://www.instagram.com/p/CqVM8-APGCs/,https...",0.192890,gpmth 🔘 artit biggbo 6 cookwithali bi enqiri...,gpmuthu 24 🔘 artist biggboss s6 cookuwithcomal...,fashion
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,2222,51954,zubaira tukhugov ufc fighter 19-4 record • воп...,35503.232,нереальный вкус @ gorillaenergi @ gorillafight...,[],[],"['дубай', 'gorillaenergy', 'gorillafighting']","https://www.instagram.com/p/CqazvpjMM5l/,https...",0.683359,zbaira tkhv fc fighter 9- record • вопросам со...,Zubaira Tukhugov UFC Fighter 19-4 Record • Wri...,parenting_family
2222,2223,5749589,zubbi michael actor http : //youtu.be/drtnohfzafa,251222.500,midnight cruis 🔥god real # zm # a1 # do # nawe...,[],[],"['zm', 'a1', 'doings', 'nawedeyhere', 'blessup...","https://www.instagram.com/p/CqMJCMXjlCF/,https...",0.043694,zbbi michael actor http : //yot.be/drtnohfzafa...,zubbi michael actor http : //youtu.be/drtnohfz...,travel
2223,2224,408734,zvezd granda tv programm grand.nova.r,28208.856,kim @ voja.nedeljkov obara ruke u zvezdama gra...,[],[],"['zvezdegranda', 'zvezdegranda']","https://www.instagram.com/p/CqV3wt4KAq-/,https...",0.069015,zvezd granda tv programm grand.nova.r kim @ vo...,Star Granda TV Programm Grand.nova.r Kim @ voj...,social
2224,2225,894950,aziz zyzz shavershian comunidad | motiv post e...,16446.000,"nan , zyzz last vlog . . 👉 @ zyzzmad follow : ...",[],[],"['zyzzpose', 'zyzz', 'zyzzmotivation', 'zyzzle...","https://www.instagram.com/p/CoppL0cIqJm/,https...",0.018376,aziz zyzz haverhian nidad | motiv pot everi |...,aziz zyzz shavershian comunidad | motiv post e...,music


In [19]:
df.to_csv('my_df_final_categories.csv')